## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import json

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Ribeira Grande,PT,38.5167,-28.7000,68.76,87,13,10.33,few clouds
1,1,Havoysund,NO,70.9963,24.6622,32.88,60,100,11.74,light snow
2,2,Ngunguru,NZ,-35.6167,174.5000,60.17,95,100,24.20,overcast clouds
3,3,Dikson,RU,73.5069,80.5464,13.87,88,100,17.90,overcast clouds
4,4,Lompoc,US,34.6391,-120.4579,66.78,90,90,15.01,mist


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 50
What is the maximum temperature you would like for your trip? 51


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df =  city_data_df.loc[(city_data_df['Max Temp'] >= min_temp) & (city_data_df['Max Temp'] <= max_temp)]
preferred_cities_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
35,35,Kamenka,RU,51.3223,42.7678,50.11,71,100,9.60,overcast clouds
135,135,Linkou,CN,45.3000,130.2833,50.05,36,0,8.14,clear sky
158,158,Lebu,CL,-37.6167,-73.6500,50.67,66,63,16.55,broken clouds
250,250,Kreminna,UA,49.0495,38.2179,50.76,71,64,5.03,broken clouds
533,533,Zhangye,CN,38.9342,100.4517,50.14,32,36,3.02,scattered clouds
611,611,Helong,CN,42.5397,128.9972,50.47,35,0,4.14,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_cities_df= preferred_cities_df.dropna()
clean_cities_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
35,35,Kamenka,RU,51.3223,42.7678,50.11,71,100,9.60,overcast clouds
135,135,Linkou,CN,45.3000,130.2833,50.05,36,0,8.14,clear sky
158,158,Lebu,CL,-37.6167,-73.6500,50.67,66,63,16.55,broken clouds
250,250,Kreminna,UA,49.0495,38.2179,50.76,71,64,5.03,broken clouds
533,533,Zhangye,CN,38.9342,100.4517,50.14,32,36,3.02,scattered clouds
611,611,Helong,CN,42.5397,128.9972,50.47,35,0,4.14,clear sky


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
35,Kamenka,RU,50.11,overcast clouds,51.3223,42.7678,
135,Linkou,CN,50.05,clear sky,45.3000,130.2833,
158,Lebu,CL,50.67,broken clouds,-37.6167,-73.6500,
250,Kreminna,UA,50.76,broken clouds,49.0495,38.2179,
533,Zhangye,CN,50.14,scattered clouds,38.9342,100.4517,
611,Helong,CN,50.47,clear sky,42.5397,128.9972,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
        
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    params['location']=f"{lat}, {lng}"

    # 6e. Make request and retrieve the JSON data from the search. 
    
    hotels=requests.get(base_url, params=params).json()
    #print(json.dumps(hotels, indent=4, sort_keys=True))
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.replace("", float("NaN"), inplace=True)
clean_hotel_df = hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
135,Linkou,CN,50.05,clear sky,45.3000,130.2833,Yueliang Resort
158,Lebu,CL,50.67,broken clouds,-37.6167,-73.6500,Hostal Las Lilas
250,Kreminna,UA,50.76,broken clouds,49.0495,38.2179,"ОЗЕРНЫЙ, САНАТОРИЙ"
533,Zhangye,CN,50.14,scattered clouds,38.9342,100.4517,Zhangye Lvzhou Holiday Inn
611,Helong,CN,50.47,clear sky,42.5397,128.9972,Helong Hotel


In [10]:
# 8a. Create the output File (CSV)
output_data_file = 'WeatherPy_vacation.csv'

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))